<a href="https://colab.research.google.com/github/thmswhelan/GAA_Rankings/blob/main/GAA_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import re
import numpy as np

In [29]:
url_rankings = 'https://github.com/thmswhelan/GAA_Rankings/raw/main/GAA%20Football%20Rankings.xlsx'
url_results = 'https://github.com/thmswhelan/GAA_Rankings/raw/main/GAA%20Football%20Results.xlsx'

# Read the Excel file into a Pandas DataFrame
df_rankings = pd.read_excel(url_rankings)
df_results = pd.read_excel(url_results)

In [30]:
df_rankings.head()

,Team,Round 0,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6,Round 7,Round 8,...,Round 12,Round 13,Round 14,Round 15,Round 16,Round 17,Round 18,Round 19,Round 20,Round 21
0,Mayo,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Down,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Waterford,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Monaghan,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kerry,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df_results.head()

,Team 1,Score 1,Score 2,Team 2,Competition,Round Number
0,Mayo,1-08,1-07,Roscommon,League,Round 1
1,Down,1-12,2-15,Laois,League,Round 1
2,Waterford,0-09,1-07,Wicklow,League,Round 1
3,Monaghan,2-13,1-13,Dublin,League,Round 1
4,Kerry,0-11,0-07,Tyrone,League,Round 1


In [57]:
def Update_Rankings(Round):

  #Filter to keep only "Round" fixtures
  df_results1 = df_results[df_results["Round Number"] == Round]

  #Convert scores into points
  df_results1[['Left 1', 'Right 1']] = df_results1['Score 1'].str.split('-', 1, expand=True)
  df_results1[['Left 2', 'Right 2']] = df_results1['Score 2'].str.split('-', 1, expand=True)

  #Convert scores to number
  df_results1['Left 1'] = pd.to_numeric(df_results1['Left 1'], errors='coerce')
  df_results1['Left 2'] = pd.to_numeric(df_results1['Left 2'], errors='coerce')
  df_results1['Right 1'] = pd.to_numeric(df_results1['Right 1'], errors='coerce')
  df_results1['Right 2'] = pd.to_numeric(df_results1['Right 2'], errors='coerce')

  df_results1['Score 1'] = df_results1['Left 1']*3 + df_results1['Right 1']
  df_results1['Score 2'] = df_results1['Left 2']*3 + df_results1['Right 2']

  columns_to_drop = ['Left 1', 'Left 2', 'Right 1', 'Right 2']
  df_results1 = df_results1.drop(columns=columns_to_drop)

  #Get the difference between teams scores
  df_results1['Difference'] = df_results1['Score 1']-df_results1['Score 2']

  match = match = re.search(r'\d+', Round)

  if match:
    extracted_number = int(match.group())
  else:
    print("No number found in the string.")

  Ranking_Round_Number = extracted_number -1

  Ranking_Round = "Round " + str(Ranking_Round_Number)

  df_rankings_round = df_rankings[["Team", Ranking_Round]]

  # Merge df_results1 based on "Team 1" and "Team 2" columns
  result_df = df_results1.merge(df_rankings_round, left_on="Team 1", right_on="Team", how="left")
  result_df = result_df.merge(df_rankings_round, left_on="Team 2", right_on="Team", how="left", suffixes=("_team1", "_team2"))
  result_df = result_df.rename(columns={Ranking_Round +"_team1": "Ranking_team1"})
  result_df = result_df.rename(columns={Ranking_Round +"_team2": "Ranking_team2"})

  result_df['diff_ranking'] = result_df['Ranking_team2'] - result_df['Ranking_team1']


  # Creating conditions
  condition1 = (result_df['Difference'] > 0) & (result_df['Difference'] <= 8)
  condition2 = (result_df['Difference'] == 0)
  condition3 = (result_df['Difference'] < 0) & (result_df['Difference'] >= -8)
  condition4 = (result_df['Difference'] > 8)
  condition5 = (result_df['Difference'] < -8)

  # Apply the condition to filter rows where 'Competition' is 'Championship'
  condition_championship = result_df['Competition'] == 'Championship'

  if condition_championship.any():
    # Calculate 'ranking_change' based on the conditions
    result_df['ranking_change'] = (np.where(condition1, result_df['diff_ranking'] * 0.1 + 1,np.where(condition2, result_df['diff_ranking'] * 0.1,np.where(condition3, result_df['diff_ranking'] * 0.1 - 1,np.where(condition4, (result_df['diff_ranking'] * 0.1 + 1)*1.5,np.where(condition5, (result_df['diff_ranking'] * 0.1 - 1)*1.5,0))))))*1.5
  else:
    result_df['ranking_change'] = np.where(condition1, result_df['diff_ranking'] * 0.1 + 1,np.where(condition2, result_df['diff_ranking'] * 0.1,np.where(condition3, result_df['diff_ranking'] * 0.1 - 1,np.where(condition4, (result_df['diff_ranking'] * 0.1 + 1)*1.5,np.where(condition5, (result_df['diff_ranking'] * 0.1 - 1)*1.5,0)))))

  #Create new team rankings
  result_df['New_Ranking_team1'] = result_df['Ranking_team1'] + result_df['ranking_change']
  result_df['New_Ranking_team2'] = result_df['Ranking_team2'] - result_df['ranking_change']

  # Extract the relevant columns from result_df
  result_df_subset = result_df[['Team 1','Team 2', 'New_Ranking_team1', 'New_Ranking_team2']]

  round_name = Round

  # Update the corresponding columns in df_rankings with the calculated rankings
  df_rankings.update(result_df_subset)

  # The df_rankings DataFrame now contains the updated rankings from result_df for "Round 1"
  print(df_rankings)

In [58]:
Update_Rankings("Round 1")

         Team  Round 0  Round 1  Round 2  Round 3  Round 4  Round 5  Round 6  \
0        Mayo       50      NaN      NaN      NaN      NaN      NaN      NaN   
1        Down       50      NaN      NaN      NaN      NaN      NaN      NaN   
2   Waterford       50      NaN      NaN      NaN      NaN      NaN      NaN   
3    Monaghan       50      NaN      NaN      NaN      NaN      NaN      NaN   
4       Kerry       50      NaN      NaN      NaN      NaN      NaN      NaN   
5      Galway       50      NaN      NaN      NaN      NaN      NaN      NaN   
6       Clare       50      NaN      NaN      NaN      NaN      NaN      NaN   
7     Kildare       50      NaN      NaN      NaN      NaN      NaN      NaN   
8   Fermanagh       50      NaN      NaN      NaN      NaN      NaN      NaN   
9       Meath       50      NaN      NaN      NaN      NaN      NaN      NaN   
10     Carlow       50      NaN      NaN      NaN      NaN      NaN      NaN   
11     Offaly       50      NaN      NaN

<ipython-input-57-41ba843d9554>:7: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_results1[['Left 1', 'Right 1']] = df_results1['Score 1'].str.split('-', 1, expand=True)
<ipython-input-57-41ba843d9554>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_results1[['Left 1', 'Right 1']] = df_results1['Score 1'].str.split('-', 1, expand=True)
<ipython-input-57-41ba843d9554>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy